<h1  style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">Stacked & Convolutional Autoencoder for Dimensionality Reduction.</h1><a id = "1" ></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background:#0096FF ; border:0' role="tab" aria-controls="home"><center>Quick Navigation</center></h3>

* [Importing Libraries](#1.1)
    
* [Data Preprocessing](#2)
    
    - [Load Training Dataset](#2.1)
    
    - [Split Data](#2.2)
    
    - [Image Preprocessing](#2.3)
    
    
* [Data Visualization](#3)

    * [Model Summary](#5)
    
    
* [**Autoencoder Architecture**](#5.3)
    
   - [**Stacked Autoencoder**](#5.3)
    
   - [**Convolutional autoencoder**](#5.5)
    
* [Save Model](#6.2)

* [Plot Evaluation](#8)

**Objective**
* The stacker Autoencoder is used to Dimensionality reduction of the given Digit images from the MNIST dataset and Fashion MNIST dataset implemented using Tensorflow 2.0.

<h1  style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">1. IMPORTING LIBRARIES</h1><a id = "1.1" ></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import decomposition

import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense, Activation,Flatten, Conv2D, Dropout,Reshape
from tensorflow.keras.layers import MaxPool2D, LSTM, BatchNormalization,concatenate, Conv2DTranspose
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.layers import ELU
from tensorflow.keras.losses import sparse_categorical_crossentropy, categorical_crossentropy
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,SGD
import warnings
warnings.filterwarnings('ignore')

from sklearn.manifold import TSNE

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h1 style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">2. DATA PREPROCESSING</h1><a id = "2" ></a>

<h1  style='color:#0096FF; background:white ; border:0' class="list-group-item list-group-item-action active">2.1 LOAD TRAINING DATASET</h1><a id = "2.1" ></a>

In [3]:

fashion_train_data = pd.read_csv('./Datasets/fashion-mnist_train.csv')


In [5]:
fashion_test_data = pd.read_csv('./Datasets/fashion-mnist_test.csv')

<h3  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">Data Dimension</h3>

Data Dimension of the Digits MNIST Dataset

In [7]:
print(f"train.csv size is {fashion_train_data.shape}")
print(f"test.csv size is {fashion_test_data.shape}")

train.csv size is (60000, 785)
test.csv size is (10000, 785)


In [8]:
train_data= fashion_train_data
test_data= fashion_test_data

In [9]:
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
train_data.describe()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.00000
mean,4.500000,0.000900,0.006150,0.035333,0.101933,0.247967,0.411467,0.805767,2.198283,5.682000,...,34.625400,23.300683,16.588267,17.869433,22.814817,17.911483,8.520633,2.753300,0.855517,0.07025
std,2.872305,0.094689,0.271011,1.222324,2.452871,4.306912,5.836188,8.215169,14.093378,23.819481,...,57.545242,48.854427,41.979611,43.966032,51.830477,45.149388,29.614859,17.397652,9.356960,2.12587
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,4.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,58.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,9.000000,16.000000,36.000000,226.000000,164.000000,227.000000,230.000000,224.000000,255.000000,254.000000,...,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,170.00000


**To Check how many pixel columns are there such that all their values are zero as we can see from the describe() function.**

Data dimension of the Fashion MNIST dataset

In [11]:
print(f"train.csv size is {fashion_train_data.shape}")
print(f"test.csv size is {fashion_test_data.shape}")

train.csv size is (60000, 785)
test.csv size is (10000, 785)


<h3  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">Missing values</h3>

In [12]:
train_data.isna().sum()

label       0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 785, dtype: int64

In [13]:
fashion_train_data.isnull().sum()

label       0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 785, dtype: int64

* No missing values in both datasets

In [14]:
train_labels=train_data['label']

<h1 style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">2.2 SPLIT DATA</h1><a id = "2.2" ></a>

In [15]:
img_rows, img_cols = 28, 28 # 28*28 dimension of image reshape
num_classes = 10 # 10 number of labels

<h1 style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">2.3 PREPROCESSING</h1><a id = "2.3" ></a>

<h3  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">One hot Encoding</h3>

One hot encoding is one method of converting data to prepare it for an algorithm and get a better prediction. With one-hot, we convert each categorical value into a new categorical column and assign a binary value of 1 or 0 to those columns. Each integer value is represented as a binary vector.

In [16]:
def data_prep(raw):
    out_y = tensorflow.keras.utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    # normalization
    out_x = x_shaped_array / 255
    return out_x, out_y

x, y = data_prep(train_data)
x_fashion,y_fashion = data_prep(fashion_train_data)

<h3  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">Data Split</h3>

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)#ratio 90:10
x_train, x_val, y_train, y_val= train_test_split(x_train, y_train, test_size = 1/9, random_state=42)

In [18]:
x_train_fashion, x_test_fashion, y_train_fashion, y_test_fashion = train_test_split(x_fashion, y_fashion, test_size=0.1, random_state=42)#ratio 90:10
x_train_fashion, x_val_fashion, y_train_fashion, y_val_fashion= train_test_split(x_train_fashion, y_train_fashion, test_size = 1/9, random_state=42)

* Splitting data into 90% training and 10% test data.
* Splitting the training data into 90% training data and 10% validation data

<h3  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">Reshaping the Data to 4 dimension</h3>

In [19]:
# Normalization
test_data = test_data / 255  
# reshaping
test_data = test_data.values.reshape(-1,28,28,1)
test_data.shape

ValueError: cannot reshape array of size 7850000 into shape (28,28,1)

In [ ]:
# Normalization
fashion_test_data = fashion_test_data / 255  
# reshaping
#fashion_test_data = fashion_test_data.values.reshape(-1,28,28,1)
#fashion_test_data.shape

<h3  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">Data Dimension</h3>

Dimensions for MNIST Digit dataset.

In [ ]:
print(f"Training data size is {x_train.shape}")
print(f"Training data size is {y_train.shape}")
print(f"Testing data size is {x_test.shape}")
print(f"Training data size is {y_test.shape}")

Dimensions of Fashion MNIST Dataset

In [ ]:
print(f"Training data size is {x_train_fashion.shape}")
print(f"Training data size is {y_train_fashion.shape}")
print(f"Testing data size is {x_test_fashion.shape}")
print(f"Training data size is {y_test_fashion.shape}")

<h1  style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">3. VISUALIZE DATA</h1><a id = "3" ></a>

In [ ]:
title=[j for i in range(1, 10) for j in range(0,10) if y_train[i][j] == 1]
title

In [ ]:
title_fashion=[j for i in range(1, 10) for j in range(0,10) if y_train_fashion[i][j] == 1]
title_fashion

**Plotting the Fashion MNIST dataset.** 

In [ ]:
plt.figure(figsize=(9,9))
for i in range(1, 10):
    plt.subplot(330 + i)
    plt.imshow(x_train_fashion[i], cmap=plt.get_cmap('gray'))
    plt.title(title_fashion[i-1])
    
plt.tight_layout()

**Plotting the Digits MNIST dataset.**

In [ ]:
plt.figure(figsize=(7,9))
for i in range(1, 10):
    plt.subplot(330 + i)
    plt.imshow(x_train[i], cmap=plt.get_cmap('gray'))
    plt.title(title[i-1])
    
plt.tight_layout()

<h1  style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">4. MODEL SUMMARY</h1><a id = "5" ></a>

<h1  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">Stacked Autoencoder Architecture</h1><a id = "5.3" ></a>

  **Stacked Autoencoder** is a method used for compressing and dimensionality reduction of the Digit images.
* Stacked Autoencoder has stacked_encoder and stacker decoder layers.  

**Stacked Encoder**

* The Stacked encoder has 3 hidden layers. Flatten layer with 100 neural units, Dense layer with 100 neurons , Dense layer with 20 neurons with 'SELU' activation functions.

**Stacked Decoder**

* The Stacked encoder has 3 hidden layers. Dense layer with 100 neural units, Dense layer with 784 neurons (activation='sigmoid') , Reshape layer with 28,28 image rows,image columns.

The Autoencoder model is one of the Dimensionality reduction technique and denoises the outliers, errors in the dataset.


In [ ]:
stacked_encoder = Sequential()

stacked_encoder.add(Flatten(input_shape=(img_rows, img_cols, 1)))
stacked_encoder.add(Dense(100,activation='selu'))
stacked_encoder.add(Dense(30,activation='selu'))

stacked_decoder=Sequential()
stacked_decoder.add(Dense(100 ,activation='selu',input_shape=[30]))
stacked_decoder.add(Dense(28*28 ,activation='sigmoid'))
stacked_decoder.add(Reshape([28,28]))

stacked_ae = Sequential([stacked_encoder,stacked_decoder])
model = stacked_ae

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='stacked_autoencoder.png', show_shapes=True)
from IPython.display import Image
Image("stacked_autoencoder.png")

<h1  style='color:#0096FF; background:#FFF ' class="list-group-item list-group-item-action active">Convolutional Autoencoder</h1><a id = "5.5" ></a>

While dealing with images, CNN's are far better suited than dense networks to work with images. So we need to build a **Convolutional autoencoder** perfectly works as an autoencoder for images. 

* The Convolutional encoder is a regular CNN composed of Convolutional layers and pooling layers. This typically reduces the spatial dimensionality of the inputs (i.e height and width) while increasing the depth(number of feature maps)

* The Convolutional decoder must do the reverse (upscale the image and reduce its depth back to original dimensions), this is transpose Convolutional layers (alternatively combining upsampling layers with convolutional layers )

In [ ]:
conv_encoder = Sequential()
conv_encoder.add(Reshape([28,28,1],input_shape=[28,28]))
conv_encoder.add(Conv2D(16,kernel_size=3,padding='same',activation='selu'))
conv_encoder.add(MaxPool2D(pool_size=2))
conv_encoder.add(Conv2D(32,kernel_size=3,padding='same',activation='selu'))
conv_encoder.add(MaxPool2D(pool_size=2))
conv_encoder.add(Conv2D(64,kernel_size=3,padding='same',activation='selu'))
conv_encoder.add(MaxPool2D(pool_size=2))

conv_decoder = Sequential()
conv_decoder.add(Conv2DTranspose(32,kernel_size=3,strides=2,padding="valid",activation='selu',input_shape=[3,3,64]))
conv_decoder.add(Conv2DTranspose(16,kernel_size=3,strides=2,padding="same",activation='selu'))
conv_decoder.add(Conv2DTranspose(1,kernel_size=3,strides=2,padding="same",activation='sigmoid'))
conv_decoder.add(Reshape([28,28]))

conv_ae = Sequential([conv_encoder,conv_decoder])

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='Conv_autoencoder.png', show_shapes=True)
from IPython.display import Image
Image("Conv_autoencoder.png")

In [ ]:
epochs = 30

# Fitting Convolutional AutoEncoder model on Fashion MNIST dataset

In [ ]:
conv_ae.compile(optimizer = SGD(lr=1.5),loss = 'binary_crossentropy')
conv_ae_fit = conv_ae.fit(x=x_train_fashion,y=x_train_fashion, epochs=epochs,validation_data=(x_val_fashion,x_val_fashion) ,verbose =1)

# Fitting Stacked Autoencoder model on Fashion MNIST dataset

In [ ]:
model.compile(optimizer = SGD(lr=1.5),loss = 'binary_crossentropy' )
model_fit = model.fit(x=x_train_fashion,y=x_train_fashion, epochs=epochs,validation_data=(x_val_fashion,x_val_fashion) ,verbose =1)

# Fitting Stacked Autoencoder model on Digits MNIST dataset

In [ ]:
stacked_ae.compile(optimizer = SGD(lr=1.5),loss = 'binary_crossentropy' )
stacked_ae_fit = stacked_ae.fit(x=x_train,y=x_train, epochs=epochs,validation_data=(x_val,x_val) ,verbose =1)

<h1  style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">SAVE MODEL</h1><a id = "6.2" ></a>

In [ ]:
!mkdir -p saved_model
conv_ae.save('saved_model/conv_ae.json')
model.save('saved_model/stacked_autoencoder_fashion_mnist.json')
stacked_ae.save('saved_model/stacked_autoencoder_digit_mnist.json')


<h1  style='color:white; background:#0096FF ; border:0;text-align: center' class="list-group-item list-group-item-action active">6. PLOT & EVALUATION</h1><a id = "8" ></a>

## Digit MNIST compression using Stacked Autoencoder

In [ ]:
x_test_compress = stacked_ae.predict(x_test)

## Digit MNIST compression using Convolutional Autoencoder

In [ ]:
x_test_convae_compress = conv_ae.predict(x_test)

## Plotting the uncompressed and compressed MNIST Digit images together using Stacked Autoencoder and Convolutional Autoencoder.

In [ ]:
fig, axs = plt.subplots(7,3,figsize=(20,30))
axs[0][0].set_title('Original image')
axs[0][1].set_title('Stacked Autoencoder compressed image')
axs[0][2].set_title('Convolutional Autoencoder image compression')
for i in range(0, 7):
    axs[i][0].imshow(x_test[i], cmap=plt.get_cmap('gray'))
    axs[i][1].imshow(x_test_compress[i], cmap=plt.get_cmap('gray'))
    axs[i][2].imshow(x_test_convae_compress[i], cmap=plt.get_cmap('gray'))
plt.tight_layout()

## Fashion MNIST compression using Stacker Autoencoder and Convolutional Autoencoder

In [ ]:
x_test_compress = model.predict(x_test_fashion)
fashion_compress = conv_ae.predict(x_test_fashion)

## Plotting the uncompressed and compressed Fashion MNIST images together.

In [ ]:
fig, axs = plt.subplots(7,3,figsize=(20,30))
axs[0][0].set_title('Original image')
axs[0][1].set_title('Stacked Autoencoder compressed image')
axs[0][2].set_title('Convolutional Autoencoder image compression')
for i in range(0, 7):
    axs[i][0].imshow(x_test_fashion[i], cmap=plt.get_cmap('gray'))
    axs[i][1].imshow(x_test_compress[i], cmap=plt.get_cmap('gray'))
    axs[i][2].imshow(fashion_compress[i], cmap=plt.get_cmap('gray'))
plt.tight_layout()
plt.show()

### Conclusion
      The Convolutional Autoencoder is better than Stacked Autoencoder when model is applied on compressing and decompressing Image datasets.

##### If you like my work, please do upvote. Thanks - `@tejasurya`